In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

dataset_train = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset_test = datasets.MNIST('../data', train=False,
                       transform=transform)

100%|██████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 33264294.60it/s]


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 32208373.79it/s]


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



100%|██████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 35019654.03it/s]

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw


100%|█████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 4481422.90it/s]


Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



In [5]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=14, metavar='N',
                    help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
                    help='learning rate (default: 1.0)')
parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
                    help='Learning rate step gamma (default: 0.7)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--no-mps', action='store_true', default=False,
                    help='disables macOS GPU training')
parser.add_argument('--dry-run', action='store_true', default=False,
                    help='quickly check a single pass')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
                    help='For Saving the current Model')
args = parser.parse_args(args=[])

In [8]:
args

Namespace(batch_size=64, test_batch_size=1000, epochs=14, lr=1.0, gamma=0.7, no_cuda=False, no_mps=False, dry_run=False, seed=1, log_interval=10, save_model=False)

In [9]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
train_loader = torch.utils.data.DataLoader(dataset_train,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset_test, **test_kwargs)


In [13]:
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295698
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.164462
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.744045
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.464460
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.384834
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.363329
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.311845
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.296390
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.430448
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.291832
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.262254
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.159498
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.252581
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.241645
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.200360
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.277170
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.504604
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.269502
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.342340
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.206135
Train Epoch: 1 [12800/60000 (

In [15]:
torch.save(model.state_dict(), "mnist_cnn.pt")